In [ ]:
import torch
import torchvision

import os
import zipfile
import matplotlib.pyplot as plt
import torch
import torchvision

import requests

from torch import nn
from torchvision import transforms

from pathlib import Path

try:
  from torchinfo import summary
except:
  print("[INFO] Couldn't find torchinfo... installing it.")
  !pip install -q torchinfo
  from torchinfo import summary


try:
  from modular import download_image_data, data_setup, train_model, save
except:
  # Get the going_modular scripts
  print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
  !git clone https://github.com/AaronJ59/Projects.git
  !mv Projects/BiteID/modular/modular modular
  !mv Projects/BiteID/models models
  from modular import download_image_data, data_setup, train_model, save
  !rm -rf Projects


[INFO] Couldn't find torchinfo... installing it.
[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.
Cloning into 'Projects'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 189 (delta 40), reused 10 (delta 1), pack-reused 54 (from 2)
Receiving objects: 100% (189/189), 109.94 MiB | 32.32 MiB/s, done.
Resolving deltas: 100% (49/49), done.
mv: cannot stat 'Projects/BiteID/models': No such file or directory


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
os.makedirs("data/food80_binary")

data_dir = Path("data")
food80_binary_dir = data_dir / "food80_binary"

!unzip /content/drive/MyDrive/food80_binary_dataset.zip -d /content/data/food80_binary

Archive:  /content/drive/MyDrive/food80_binary_dataset.zip
   creating: /content/data/food80_binary/test/
   creating: /content/data/food80_binary/train/
   creating: /content/data/food80_binary/test/0_non-food/
   creating: /content/data/food80_binary/test/1_food/
  inflating: /content/data/food80_binary/test/1_food/3870085.jpg  
  inflating: /content/data/food80_binary/test/1_food/914304.jpg  
  inflating: /content/data/food80_binary/test/1_food/1259396.jpg  
  inflating: /content/data/food80_binary/test/1_food/3146827.jpg  
  inflating: /content/data/food80_binary/test/1_food/1016430.jpg  
  inflating: /content/data/food80_binary/test/1_food/538282.jpg  
  inflating: /content/data/food80_binary/test/1_food/453657.jpg  
  inflating: /content/data/food80_binary/test/1_food/1122003.jpg  
  inflating: /content/data/food80_binary/test/1_food/146833.jpg  
  inflating: /content/data/food80_binary/test/1_food/1930366.jpg  
  inflating: /content/data/food80_binary/test/1_food/1671002.jpg  
 

In [ ]:
os.mkdir("data/food80_multiclass")

food80_multiclass_dir = data_dir / "food80_multiclass"

!unzip /content/drive/MyDrive/food80_multiclass_dataset.zip -d /content/data/food80_multiclass

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/food80_multiclass/test/fish_and_chips/1784894.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/3860849.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/2799763.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/3388814.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/1662738.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/3912593.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/3909172.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/4128.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/2497717.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/1830225.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/2065591.jpg  
  inflating: /content/data/food80_multiclass/test/fish_and_chips/3521924.jpg  
  in

In [ ]:
def create_effnetb2_model(num_classes: int,
                          seed:int=42):


  # Import efficientnetb2

  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit 5 classes
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
  )

  return effnetb2, effnetb2_transform




effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=80)

# Create effnetb2 model for binary classifier
effnetb2_binary, effnetb2_transform = create_effnetb2_model(num_classes=1)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 121MB/s]


In [ ]:
effnetb2.load_state_dict(
    torch.load(f="drive/MyDrive/effnetb2_food80_multiclass_model_65_percent_acc.pth",
               map_location=torch.device("cpu"))
)

<ipython-input-8-30dc2ffa1a8a>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f="drive/MyDrive/effnetb2_food80_multiclass_model_65_percent_acc.pth",


<All keys matched successfully>

In [ ]:
effnetb2_binary.load_state_dict(
    torch.load(f="drive/MyDrive/effnetb2_binary_food80_model.pth",
               map_location=torch.device("cpu"))
)

<ipython-input-9-4279e5a7e0e0>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f="drive/MyDrive/effnetb2_binary_food80_model.pth",


<All keys matched successfully>

In [ ]:
test_dir = Path("data/food80_multiclass/test")

image_paths = list(Path(test_dir).glob("*/*.jpg"))


binary_class_names = ["0_non-food", "1_food"]

In [ ]:
# To get multiclass class names
train_dataloader, test_dataloader, multiclass_class_names = data_setup.create_dataloaders(train_dir="data/food80_multiclass/train",
                                                                               test_dir=test_dir,
                                                                               transform=effnetb2_transform,
                                                                               batch_size=32)


In [ ]:
import pathlib
import torch

from PIL import Image
from timeit import default_timer as timer
from tqdm.auto import tqdm
from typing import List, Dict

def pred_and_store(image_paths: List[pathlib.Path],
                   model: torch.nn.Module,
                   transform: torchvision.transforms,
                   device: str = "cuda" if torch.cuda.is_available() else "cpu") -> List[Dict]:

  pred_list = []

  model.to(device)

  for path in tqdm(image_paths):

    pred_dict = {}

    class_name = path.parent.stem
    pred_dict["class"] = class_name

    pred_dict["path"] = path

    start_time = timer()

    image = Image.open(path)

    # Transform the image, add batch dimension and put image on target device
    image_transformed = transform(image).unsqueeze(dim=0).to(device)

    model.eval()

    with torch.inference_mode():
      pred_logits = model(image_transformed)

      end_time = timer()


      pred_probs = torch.softmax(pred_logits, dim=1)
      pred_label = torch.argmax(pred_probs, dim=1)

      pred_dict["pred_prob"] = round(pred_probs.max().cpu().item(), 4)
      pred_dict["pred_class"] = multiclass_class_names[pred_label.cpu()]



    pred_dict["pred_time"] = round(end_time - start_time, 4)
    pred_dict["correct_pred"] = class_name == multiclass_class_names[pred_label.cpu()]

    # Append created dictionary to the list
    pred_list.append(pred_dict)

  return pred_list




In [ ]:
effnetb2_pred_list = pred_and_store(image_paths=image_paths,
                                   model=effnetb2,
                                   transform=effnetb2_transform,
                                    device=device)

  0%|          | 0/14000 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
effnetb2_pred_df = pd.DataFrame(effnetb2_pred_list)

effnetb2_pred_df.head()

,class,path,pred_prob,pred_class,pred_time,correct_pred
0,frozen_yogurt,data/food80_multiclass/test/frozen_yogurt/7357...,0.6834,frozen_yogurt,1.3232,True
1,frozen_yogurt,data/food80_multiclass/test/frozen_yogurt/9718...,0.3497,frozen_yogurt,0.0421,True
2,frozen_yogurt,data/food80_multiclass/test/frozen_yogurt/3622...,0.6723,frozen_yogurt,0.0359,True
3,frozen_yogurt,data/food80_multiclass/test/frozen_yogurt/2738...,0.7765,frozen_yogurt,0.0368,True
4,frozen_yogurt,data/food80_multiclass/test/frozen_yogurt/7711...,0.9894,frozen_yogurt,0.0373,True


In [ ]:
effnetb2_pred_df.correct_pred.value_counts()

,count
correct_pred,
True,9203
False,4797


## Implement Gradio interface

In [ ]:
import gradio as gr

In [ ]:
import time

effnetb2_binary.to(device)
effnetb2.to(device)


def image_classification(image):



  start = time.time()

  # First, pass image through binary classifier to see if the image is food
  image_transformed = effnetb2_transform(image).unsqueeze(dim=0).to(device)

  effnetb2_binary.eval()

  with torch.inference_mode():
    pred_logit = effnetb2_binary(image_transformed)

    pred_prob = torch.sigmoid(pred_logit)
    pred_label = (pred_prob >= 0.5).long()
    print(pred_label)
    # if the image is food:
    if binary_class_names[pred_label] == "1_food":

      image_transformed = effnetb2_transform(image).unsqueeze(0).to(device) # Adding a batch dimension

      effnetb2.eval()

      with torch.inference_mode():
        pred_logits = effnetb2(image_transformed)

        pred_probs = torch.softmax(pred_logits, dim=1)

        pred_label = torch.argmax(pred_probs, dim=1)

        # Dictionary holds the class names and its given probability by the model
      pred_dict = {multiclass_class_names[i]: float(pred_probs[0][i]) for i in range(len(multiclass_class_names))}

      end = time.time()
      # Time it took to make the prediction
      time_to_run = round(end - start, 4)


      return pred_dict, time_to_run



    else:
      return "Food undetected", "Food undetected"



In [ ]:
binary_class_names = ["0_non-food", "1_food"]

test_dir = Path("data/food80_multiclass/test")

import random
from PIL import Image
## Get a random image
image_paths = list(Path(test_dir).glob("*/*.jpg"))

image_path = random.sample(image_paths, k=1)[0]
print(image_path)

img = Image.open(image_path)

pred_dict, time_to_run = image_classification(img)

pred_dict, time_to_run

data/food80_multiclass/test/peking_duck/2123999.jpg
tensor([[1]])


({'apple_pie': 0.00036394785274751484,
  'baby_back_ribs': 0.0019380153389647603,
  'beet_salad': 0.0014148068148642778,
  'beignets': 0.04937928915023804,
  'bibimbap': 0.0038352878764271736,
  'breakfast_burrito': 0.09402184188365936,
  'bruschetta': 0.001793628092855215,
  'caesar_salad': 0.0007210217299871147,
  'cannoli': 0.001169320777989924,
  'caprese_salad': 0.0024332001339644194,
  'carrot_cake': 0.00014313659630715847,
  'ceviche': 0.003282445017248392,
  'cheesecake': 0.00018003997683990747,
  'chicken_curry': 0.0005303784855641425,
  'chicken_quesadilla': 0.057228054851293564,
  'chicken_wings': 0.00020284006313886493,
  'chocolate_cake': 0.00017889257287606597,
  'chocolate_mousse': 6.012850280967541e-05,
  'churros': 0.00028414413100108504,
  'clam_chowder': 0.0009051093365997076,
  'club_sandwich': 0.015215323306620121,
  'crab_cakes': 0.013088677078485489,
  'creme_brulee': 6.0013928305124864e-05,
  'croque_madame': 0.004069852642714977,
  'cup_cakes': 0.00046736779040

In [ ]:
import random

examples = random.sample(image_paths, k=4)
examples

[PosixPath('data/food80_multiclass/test/chocolate_cake/1319076.jpg'),
 PosixPath('data/food80_multiclass/test/seaweed_salad/3174875.jpg'),
 PosixPath('data/food80_multiclass/test/spaghetti_bolognese/78301.jpg'),
 PosixPath('data/food80_multiclass/test/prime_rib/9053.jpg')]

In [ ]:
# Picked these out of the random.sample choices. Just for consistency
examples = ["data/food80_multiclass/test/garlic_bread/133323.jpg", "data/food80_multiclass/test/mussels/3033844.jpg", "data/food80_multiclass/test/miso_soup/1023581.jpg", "data/food80_multiclass/test/prime_rib/9053.jpg"]

In [ ]:
img = Image.open(examples[0])

pred_dict, time_to_run = image_classification(img)

pred_dict, time_to_run

tensor([[1]], device='cuda:0')


({'apple_pie': 0.023519573733210564,
  'baby_back_ribs': 0.0002247245138278231,
  'beet_salad': 0.06319569796323776,
  'beignets': 0.0002500550472177565,
  'bibimbap': 0.0050712027586996555,
  'breakfast_burrito': 0.0018767154542729259,
  'bruschetta': 0.008062670938670635,
  'caesar_salad': 0.00035294535337015986,
  'cannoli': 0.01717309094965458,
  'caprese_salad': 0.048431240022182465,
  'carrot_cake': 0.021046428009867668,
  'ceviche': 0.00010864553041756153,
  'cheesecake': 0.0020785077940672636,
  'chicken_curry': 0.040993329137563705,
  'chicken_quesadilla': 0.03395566716790199,
  'chicken_wings': 0.00021140369062777609,
  'chocolate_cake': 0.005908583290874958,
  'chocolate_mousse': 0.015888238325715065,
  'churros': 3.5179655242245644e-05,
  'clam_chowder': 0.0019962582737207413,
  'club_sandwich': 0.0004245312593411654,
  'crab_cakes': 0.004298160318285227,
  'creme_brulee': 0.0007067397818900645,
  'croque_madame': 0.0008946339949034154,
  'cup_cakes': 0.0001079075300367549,

In [ ]:
demo = gr.Interface(fn=image_classification,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=5, label="Predictions"),
                             gr.Label(label="Prediction Time (s)")],
                    examples=examples,
                    title="BiteIDv2",
                    description="Change this")

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8915a72d942c9cf4ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Creating Gradio app structure

### Creating demos folder

In [ ]:
import shutil
from pathlib import Path

biteidv2_demo_path = Path("demos/biteidv2/")

if biteidv2_demo_path.exists():
  shutil.rmtree(biteidv2_demo_path)

biteidv2_demo_path.mkdir(parents=True,
                       exist_ok=True)

### Creating example images folder

In [ ]:
demo_examples_path = biteidv2_demo_path / "examples"
demo_examples_path.mkdir(parents=True, exist_ok=True)

for example in examples:
  shutil.copy(example, demo_examples_path)

### Moving models to demo folder

In [ ]:

try:
  shutil.copy("drive/MyDrive/effnetb2_food80_multiclass_model_65_percent_acc.pth", "demos/biteid")
  shutil.copy("drive/MyDrive/effnetb2_binary_food80_model.pth", "demos/biteid")
except:
  print(f"Models have already been moved or they haven't been uploaded")

Models have already been moved or they haven't been uploaded


### Creating model.py

In [ ]:
%%writefile demos/biteidv2/model.py

def create_effnetb2_model(num_classes: int,
                          seed:int=42):


  # Get the weights, transforms of the model and make the model
  efficientnetb2_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
  effnetb2_transform = efficientnetb2_weights.transforms()
  effnetb2 = torchvision.models.efficientnet_b2(weights=efficientnetb2_weights)

  # Freeze the base layers of efficientnetb2
  for param in effnetb2.parameters():
    param.requires_grad = False



  # Change classifier head of effnetb2 to suit 5 classes
  torch.manual_seed(seed)
  effnetb2.classifier = nn.Sequential(
      nn.Dropout(p=0.3, inplace=True),
      nn.Linear(in_features=1408, out_features=num_classes),
  )

  return effnetb2, effnetb2_transform

Writing demos/biteidv2/model.py


### Creating app.py

In [ ]:
%%writefile demos/biteidv2/app.py

from model import create_effnetb2_model

import time
import gradio as gr
import torch

import os
import torch

# class names for multiclass
multiclass_class_names = ['apple_pie', 'baby_back_ribs', 'beet_salad', 'beignets', 'bibimbap', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'dumplings', 'edamame', 'eggs_benedict', 'falafel', 'filet_mignon', 'fish_and_chips', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'frozen_yogurt', 'garlic_bread', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'hamburger', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'macaroni_and_cheese', 'miso_soup', 'mussels', 'omelette', 'onion_rings', 'pad_thai', 'paella', 'pancakes', 'panna_cotta', 'peking_duck', 'pho', 'pizza', 'pork_chop', 'poutine', 'prime_rib', 'ramen', 'ravioli', 'risotto', 'samosa', 'scallops', 'seaweed_salad', 'shrimp_and_grits', 'spaghetti_bolognese', 'spaghetti_carbonara', 'spring_rolls', 'steak', 'strawberry_shortcake', 'sushi', 'tacos', 'takoyaki', 'tiramisu', 'tuna_tartare', 'waffles']


binary_class_names = ["0_non-food", "1_food"]

effnetb2, effnetb2_transform = create_effnetb2_model(num_classes=80)

effnetb2_binary, effnetb2_transform = create_effnetb2_model(num_classes=1)


# Load state dicts
effnetb2.load_state_dict(
    torch.load(f="effnetb2_food80_multiclass_model_65_percent_acc.pth",
               map_location=torch.device("cpu")
               )
    )


effnetb2_binary.load_state_dict(
    torch.load(
        f="effnetb2_binary_food80_model.pth",
        map_location=torch.device("cpu")
    )
)



def image_classification(image):


  start = time.time()

  # First, pass image through binary classifier to see if the image is food
  image_transformed = effnetb2_transform(image).unsqueeze(dim=0).to(device)

  effnetb2_binary.eval()

  with torch.inference_mode():
    pred_logit = effnetb2_binary(image_transformed)

    pred_prob = torch.sigmoid(pred_logit)
    pred_label = (pred_prob >= 0.5).long()
    print(pred_label)
    # if the image is food:
    if binary_class_names[pred_label] == "1_food":

      image_transformed = effnetb2_transform(image).unsqueeze(0).to(device) # Adding a batch dimension

      effnetb2.eval()

      with torch.inference_mode():
        pred_logits = effnetb2(image_transformed)

        pred_probs = torch.softmax(pred_logits, dim=1)

        pred_label = torch.argmax(pred_probs, dim=1)

        # Dictionary holds the class names and its given probability by the model
      pred_dict = {multiclass_class_names[i]: float(pred_probs[0][i]) for i in range(len(multiclass_class_names))}

      end = time.time()
      # Time it took to make the prediction
      time_to_run = round(end - start, 4)


      return pred_dict, time_to_run

    else:
      return "Image unrecognizable", "Image unrecognizable"


examples = [["examples/" + example] for example in os.listdir("examples")]


demo = gr.Interface(fn=image_classification,
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=5, label="Predictions"),
                             gr.Label(label="Prediction Time (s)")],
                    examples=examples,
                    title="BiteIDv2",
                    description="An app that first detects if food is in the image and then classifies the image of food into one of the 80 classes")

# Launch the demo!
demo.launch()


Overwriting demos/biteidv2/app.py


### Creating requirements.txt

In [ ]:
%%writefile demos/biteidv2/requirements.txt
torch
gradio==5.9.1
torchvision==0.20.1

Writing demos/biteidv2/requirements.txt


In [ ]:
# Change into and zip the biteid folder. Everything after -x is excluded from being zipped
!cd demos/biteidv2 && zip -r ../biteidv2.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

from google.colab import files
files.download("demos/biteidv2.zip")


  adding: app.py (deflated 59%)
  adding: effnetb2_binary_food80_model.pth (deflated 8%)
  adding: effnetb2_food80_multiclass_model_65_percent_acc.pth (deflated 8%)
  adding: examples/ (stored 0%)
  adding: examples/1023581.jpg (deflated 2%)
  adding: examples/3033844.jpg (deflated 1%)
  adding: examples/9053.jpg (deflated 21%)
  adding: examples/133323.jpg (deflated 0%)
  adding: model.py (deflated 51%)
  adding: requirements.txt (deflated 5%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>